# 1. Подготовка

Проверим CPU

In [1]:
!lscpu |grep 'Model name'

Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz


Установка библиотеки для распознавания речи

In [2]:
!pip install SpeechRecognition==3.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 1.2 MB/s 


Импорт зависимостей

In [79]:
import time as t
import os
import speech_recognition as sr

from evaluate import load 

In [4]:
!gdown 1BOoXkBL1FgrFO_7M_BMKmDduqij16maV
!gdown 1GSUx2b2V2iapbW79InaIqtZUsIMDgWq2

Downloading...
From: https://drive.google.com/uc?id=1BOoXkBL1FgrFO_7M_BMKmDduqij16maV
To: /content/test.wav
100% 334k/334k [00:00<00:00, 66.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1GSUx2b2V2iapbW79InaIqtZUsIMDgWq2
To: /content/tes2.wav
100% 217k/217k [00:00<00:00, 89.9MB/s]


# 2. Распознование текста

In [6]:
r = sr.Recognizer()

Функция для распознавания текста в аудиофайле

In [68]:
def transript_file(file_path, language):
  sample = sr.WavFile(file_path)
  start_time = t.time()
  with sample as audio:
    content = r.record(audio)
    r.adjust_for_ambient_noise(audio)
  try:
    text = r.recognize_google(content, language=language)
    end_time = round((t.time() - start_time), 3)
    return text, end_time
  except Exception as e:
    print("Возникла ошибка в процессе распознавания. Ошибка: " + str(e))


In [73]:
text_1, work_time_1 = transript_file('/content/test.wav', 'ru-RU')
print(f'Текст: {text_1}\nЗатраченное время: {work_time_1} секунды')

Текст: раз-два-три проверка связи 32 раз Как слышно Приём
Затраченное время: 0.621 секунды


In [74]:
text_2, work_time_2 = transript_file('/content/tes2.wav', 'ru-RU')
print(f'Текст: {text_2}\nЗатраченное время: {work_time_2} секунды')

Текст: так новая попытка попробую уговорить быстро если он невнятно возможно
Затраченное время: 2.892 секунды


# 3. Оценка распознавания

Для оценки распознавания воспользуемся метрикой WER

0 - идеальное значение

In [80]:
wer = load("wer")

In [87]:
predictions = [text_1]
references = ["раз два три проверка связи три два раз как слышно прием"]
wer_score = wer.compute(predictions=predictions, references=references)
print(f'Результат первого распознавания: {wer_score}')

Результат первого распознавания: 0.6363636363636364


In [88]:
predictions = [text_2]
references = ["так новая попытка попробую поговорить быстро и совершенно невнятно посмотрим как она справится с этой ерундой"]
wer_score = wer.compute(predictions=predictions, references=references)
print(f'Результат второго распознавания: {wer_score}')

Результат второго распознавания: 0.625


# 4. Вывод

Для медленной речи метрика получилась плохой, но это связано с дополнительными знаками припенания и текстовым представлением цифр, в тексте, который вернула модель.

С быстрой речью speech_recognition справилась плохо, часть слов было не распознано